In [23]:
import pandas as pd

In [24]:
df = pd.read_csv("../data/raw/kaggle_cancer_patients_raw.csv")

In [25]:
df.describe()

,Age,Year,Genetic_Risk,Air_Pollution,Alcohol_Use,Smoking,Obesity_Level,Treatment_Cost_USD,Survival_Years,Target_Severity_Score
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,54.421540,2019.480520,5.001698,5.010126,5.010880,4.989826,4.991176,52467.298239,5.006462,4.951207
std,20.224451,2.871485,2.885773,2.888399,2.888769,2.881579,2.894504,27363.229379,2.883335,1.199677
min,20.000000,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5000.050000,0.000000,0.900000
25%,37.000000,2017.000000,2.500000,2.500000,2.500000,2.500000,2.500000,28686.225000,2.500000,4.120000
50%,54.000000,2019.000000,5.000000,5.000000,5.000000,5.000000,5.000000,52474.310000,5.000000,4.950000
75%,72.000000,2022.000000,7.500000,7.500000,7.500000,7.500000,7.500000,76232.720000,7.500000,5.780000
max,89.000000,2024.000000,10.000000,10.000000,10.000000,10.000000,10.000000,99999.840000,10.000000,9.160000


In [26]:
df.head()

,Patient_ID,Age,Gender,Country_Region,Year,Genetic_Risk,Air_Pollution,Alcohol_Use,Smoking,Obesity_Level,Cancer_Type,Cancer_Stage,Treatment_Cost_USD,Survival_Years,Target_Severity_Score
0,PT0000000,71,Male,UK,2021,6.4,2.8,9.5,0.9,8.7,Lung,Stage III,62913.44,5.9,4.92
1,PT0000001,34,Male,China,2021,1.3,4.5,3.7,3.9,6.3,Leukemia,Stage 0,12573.41,4.7,4.65
2,PT0000002,80,Male,Pakistan,2023,7.4,7.9,2.4,4.7,0.1,Breast,Stage II,6984.33,7.1,5.84
3,PT0000003,40,Male,UK,2015,1.7,2.9,4.8,3.5,2.7,Colon,Stage I,67446.25,1.6,3.12
4,PT0000004,43,Female,Brazil,2017,5.1,2.8,2.3,6.7,0.5,Skin,Stage III,77977.12,2.9,3.62


In [27]:
df.columns

Index(['Patient_ID', 'Age', 'Gender', 'Country_Region', 'Year', 'Genetic_Risk',
       'Air_Pollution', 'Alcohol_Use', 'Smoking', 'Obesity_Level',
       'Cancer_Type', 'Cancer_Stage', 'Treatment_Cost_USD', 'Survival_Years',
       'Target_Severity_Score'],
      dtype='object')

In [28]:
df['Country_Region'].unique()

array(['UK', 'China', 'Pakistan', 'Brazil', 'Germany', 'Canada', 'USA',
       'India', 'Australia', 'Russia'], dtype=object)

In [29]:
# Filter only some countries
df = df[df['Country_Region'].isin(['UK', 'Germany', 'Canada', 'USA', 'Australia', 'Brazil'])]

In [30]:
# Replacing Stage Description (Eg: "Stage III" for "3")
df['Cancer_Stage'] = df['Cancer_Stage'].str.replace('Stage ', '', regex=True)

stage_to_int_map = {
    "0": 0, "I": 1, "II": 2, "III": 3, "IV": 4, "V": 5,
    "VI": 6, "VII": 7, "VIII": 8, "IX": 9, "X": 10,
}

df['Cancer_Stage'] = df['Cancer_Stage'].map(stage_to_int_map)

df['Cancer_Stage'].unique()

array([3, 1, 4, 0, 2])

In [31]:
# Filtering out columns

filtered_columns = ['Age', 'Gender', 'Country_Region', 'Genetic_Risk', 'Air_Pollution', 'Alcohol_Use', 'Smoking', 'Obesity_Level', 'Cancer_Type', 'Cancer_Stage', 'Survival_Years', 'Treatment_Cost_USD']

df = df[filtered_columns]

df.to_csv("../data/processed/kaggle_cancer_patients_processed.csv", index=False)

df.head()

,Age,Gender,Country_Region,Genetic_Risk,Air_Pollution,Alcohol_Use,Smoking,Obesity_Level,Cancer_Type,Cancer_Stage,Survival_Years,Treatment_Cost_USD
0,71,Male,UK,6.4,2.8,9.5,0.9,8.7,Lung,3,5.9,62913.44
3,40,Male,UK,1.7,2.9,4.8,3.5,2.7,Colon,1,1.6,67446.25
4,43,Female,Brazil,5.1,2.8,2.3,6.7,0.5,Skin,3,2.9,77977.12
5,22,Male,Germany,9.5,6.4,3.3,3.9,5.1,Cervical,4,9.5,33468.99
6,41,Male,Canada,5.1,8.2,0.3,3.7,2.1,Cervical,0,1.0,9790.83
